### Импорт библиотек

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score, train_test_split

### Загрузка и подготовка данных 

In [3]:
DATA_PATH = '../data/'
df = pd.read_excel( DATA_PATH + 'Путилково_2. Дома с метражом.xlsx', thousands='.')

In [4]:
df.head()

Проект  Номер дома  Номер квартиры  Тип планировки  Тип планировки 2  \
0  Путилково           7               1  2-46-У-ДС(1.1)               NaN   
1  Путилково           7               2  1-41-У-НС(1.2)               NaN   
2  Путилково           7               3   1-34-Л-Н(1.3)               NaN   
3  Путилково           7               4   1-33-Л-Н(1.4)               NaN   
4  Путилково           7               5  2-58-Р-ДН(1.5)               NaN   

   Общая площадь, кв,м,                               Ссылка на планировку  \
0                  44.2  https://drive.google.com/open?id=1Mi3B3SxvnxMR...   
1                  38.8  https://drive.google.com/open?id=1vbBwmuudWNen...   
2                  32.4  https://drive.google.com/open?id=1VzQwjr3faKd4...   
3                  32.6  https://drive.google.com/open?id=1hja7rCuj9qbz...   
4                  55.6  https://drive.google.com/open?id=1QrJHv9dMA54N...   

  Комнатность       Тип  Паркинг  ...    Ценазакв,м,  Ценазапомещение,тг.  \
0        2ккв  квартира      NaN  ...  100602.000000            4446608.4   
1     2ккв(с)  квартира      NaN  ...  128068.000000            4969038.4   
2        1ккв  квартира      NaN  ...   99855.345679            3235313.2   
3        1ккв  квартира      NaN  ...   96638.699387            3150421.6   
4        2ккв  квартира      NaN  ...   98904.482014            5499089.2   

     Статус  Дополнительный статус Дата старта продаж        Дата продажи  \
0   Продажа                    NaN                NaN 2019-08-29 13:12:32   
1  Свободно                    NaN                NaN                 NaT   
2   Продажа                    NaN                NaN 2018-12-10 00:00:00   
3   Продажа                    NaN                NaN 2018-12-10 00:00:00   
4   Продажа                    NaN                NaN 2018-12-10 00:00:00   

   ProfitbaseId ProfitbaseHomeId  ComplexId Площадь гардеробных  
0           NaN              NaN        NaN                 NaN  
1           NaN              NaN        NaN                 NaN  
2           NaN              NaN        NaN                 2.7  
3           NaN              NaN        NaN                 2.2  
4           NaN              NaN        NaN                 2.7  

[5 rows x 57 columns]

### Разделение выборки и нормализация

In [ ]:
# Разбиение выборки, используя перемешивание
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True, stratify=y)

In [ ]:
print (X_train.shape, X_test.shape)

In [ ]:
# Производим нормализацию признаков
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

### 1. LightGBM Regressor

In [ ]:
params = {'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'regression',#Строим регрессию
          'metric': 'l1', #встает вопрос а какую меру точности брать? mape,mae, rmse?
          'max_depth': -1,
          'num_leaves': 35,
          'feature_fraction': 0.8, #.75
          'bagging_fraction': 0.8, #.75
          'bagging_freq': 10,
          'learning_rate': 0.02,
          'num_threads': 2,
          'verbose': 1,
          'seed': 2018,
          'subsample_for_bin': 200,
          'subsample': 1,
          'max_bin': 512,
          'subsample_freq': 1,
          'min_split_gain': 0.5,
          'min_child_weight': 1,
          'min_child_samples': 5,
          'scale_pos_weight': 1}
          #'is_unbalance': True,
          #'scale_pos_weight':scale_pos_weight}

In [ ]:
gridParams = {
    'learning_rate': [0.005, 0.1, 0.01, 0.05],
    'n_estimators': [50, 100, 500],
    'num_leaves': [6,8,12,16],
    'boosting_type' : ['gbdt'],
    'objective' : ['regression'],
    'random_state' : [1,42,2018], # Updated from 'seed'
    'colsample_bytree' : [0.1, 0.5, 1],
    'subsample' : [0.1, 0.5, 1],
    'reg_alpha' : [1,0.1,0.5,0.8],
    'reg_lambda' : [1,0.1,0.5,0.8],
    }

In [ ]:
mdl = lgb.LGBMRegressor(boosting_type= 'gbdt',
          objective = 'regression',
          n_jobs = -1, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'],
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'],
          subsample_freq = params['subsample_freq'],
          min_split_gain = params['min_split_gain'],
          min_child_weight = params['min_child_weight'],
          min_child_samples = params['min_child_samples'],
          scale_pos_weight = params['scale_pos_weight'])

In [ ]:
grid = RandomizedSearchCV(
  estimator=mdl,
  param_distributions=gridParams,
  cv=5,
  scoring="neg_root_mean_squared_error",
  n_jobs=-1,
  verbose=1,
n_iter=100, random_state=42
)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
lgb_model = lgb.LGBMRegressor(**grid.best_params_)

In [ ]:
lgb_model.fit(X_train, y_train, eval_set=(X_test, y_test))

In [ ]:
lgb_model.fit(X_train, y_train)

In [ ]:
lgb_model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
print('LightGBM MAE: {}'.format(mae(y_test, lgb_model.predict(X_test))))

In [ ]:
from sklearn.metrics import mean_squared_error as rmse
print('LightGBM RMSE: {}'.format(rmse(y_test, lgb_model.predict(X_test), squared=False)))

In [ ]:
print('LightGBM MAPE: {}'.format( np.mean(abs(y_test- lgb_model.predict(X_test))/y_test)) )

In [ ]:
 lgb_model.feature_importances_

#### Вариант 2 

In [ ]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['rmse'],
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 100000,
    "n_estimators": 1000
}

In [ ]:
gbm = lgb.LGBMRegressor(**hyper_params)

In [ ]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='rmse',
        early_stopping_rounds=1000)

In [ ]:
y_pred = gbm.predict(X_train, num_iteration=gbm.best_iteration_)

In [ ]:
# Basic RMSE
print('The rmse of prediction is:', round(mean_squared_log_error(y_pred, y_train) ** 0.5, 5))

In [ ]:
test_pred = np.expm1(gbm.predict(df_test, num_iteration=gbm.best_iteration_))

### 2. CatBoost Regressor

In [ ]:
cb_model = CatBoostRegressor(verbose=True, iterations=6000, 
                              eval_metric = 'RMSE', learning_rate = 0.001,
                              depth = 10)

In [ ]:
cb_model.fit(X_train, y_train, eval_set=(X_test, y_test))

In [ ]:
cb_prediction = cb_model.predict(X_test)

In [ ]:
cb_prediction_series = pd.Series(cb_prediction)

In [ ]:
from sklearn.metrics import mean_squared_error
print('Catboost RMSE: {}'.format(mean_squared_error(y_test, cb_prediction_series, squared=False)))

In [ ]:
params = {'iterations': [1500], 
          'depth': [2,3,4,5,6,7,8,9,10,11,12],
          #'loss_function': ['Logloss'],
          #'verbose': [50],
          'eval_metric': ['RMSE'], 
          'learning_rate': [0.001, 0.01, 0.1, 1]
         }

In [ ]:
cb_model = CatBoostRegressor()

In [ ]:
clf = GridSearchCV(cb_model, params, cv=5)

In [ ]:
clf

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
clf.best_score_

In [ ]:
cb_model = CatBoostRegressor(**clf.best_params_)

In [ ]:
cb_model.fit(X_train, y_train, eval_set=(X_test, y_test))

In [ ]:
cb_model.predict(X_test)

In [ ]:
cb_prediction = cb_model.predict(X_test)

In [ ]:
cb_prediction_series = pd.Series(cb_prediction)

In [ ]:
from sklearn.metrics import mean_squared_error
print('Catboost RMSE: {}'.format(mean_squared_error(y_test, cb_prediction_series, squared=False)))